In [46]:
# !pip install pandas numpy seaborn tensorflow keras PIL sklearn 

In [ ]:
# in case you dont have the below libraries, uncomment and run the above cell to download them

# import the libraries

In [2]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import load_img, img_to_array 
from tensorflow.keras.models import load_model
import PIL


# data collection

In [3]:


width = 256
height = 256

datagen = image.ImageDataGenerator()
training_data = datagen.flow_from_directory("xray_dataset_covid19/train/",target_size=(width, height), class_mode="binary")
testing_data = datagen.flow_from_directory("xray_dataset_covid19/test/",target_size=(width, height), class_mode="binary")



batchX, batchy = training_data.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 148 images belonging to 2 classes.
Found 40 images belonging to 2 classes.
Batch shape=(32, 256, 256, 3), min=0.000, max=255.000


In [4]:
print(training_data.dtype)

float32


## Reset the  variables in case(just a precaution)

In [5]:
training_data.reset()
testing_data.reset()

# build  the model

In [6]:


model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape = (255, 255, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, 3, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, 3, 3))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

In [8]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=["mse", "accuracy","mae" ])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 85, 85, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 85, 85, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 42, 42, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 64)          1

# train the model

In [9]:


model.fit(training_data, epochs = 10, batch_size=30)
cnn = model.save_weights("simplecnn.h5")

Epoch 1/10
5/5 [==============================] - 6s 560ms/step - loss: 20.5144 - mse: 0.5067 - accuracy: 0.4797 - mae: 0.5188
Epoch 2/10
5/5 [==============================] - 4s 743ms/step - loss: 4.0179 - mse: 0.4083 - accuracy: 0.5405 - mae: 0.4512
Epoch 3/10
5/5 [==============================] - 4s 784ms/step - loss: 2.7154 - mse: 0.4169 - accuracy: 0.5541 - mae: 0.4702
Epoch 4/10
5/5 [==============================] - 4s 698ms/step - loss: 1.6663 - mse: 0.3671 - accuracy: 0.5608 - mae: 0.4409
Epoch 5/10
5/5 [==============================] - 4s 859ms/step - loss: 1.4384 - mse: 0.3457 - accuracy: 0.5811 - mae: 0.4299
Epoch 6/10
5/5 [==============================] - 4s 643ms/step - loss: 2.0027 - mse: 0.3826 - accuracy: 0.5473 - mae: 0.4536
Epoch 7/10
5/5 [==============================] - 4s 843ms/step - loss: 1.1396 - mse: 0.3277 - accuracy: 0.5676 - mae: 0.4320
Epoch 8/10
5/5 [==============================] - 4s 604ms/step - loss: 1.0416 - mse: 0.2990 - accuracy: 0.5878 - mae

# Evaluate perfomance of the model

In [7]:
model.evaluate(testing_data)

2/2 [==============================] - 0s 86ms/step - loss: 0.2494 - mse: 0.0697 - accuracy: 0.9500 - mae: 0.1956


[0.24938416481018066,
 0.06967302411794662,
 0.949999988079071,
 0.19562752544879913]

# Testing with random data

In [10]:
img = load_img("xray_dataset_covid19/test/PNEUMONIA/SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0.jpeg",
              target_size=(255, 255))

img = img.resize((255,255))
img = img_to_array(img)
img = img.reshape( (1, 255, 255, 3))
img.shape


prediction = model.predict(img)


if prediction >= 0.5:
    print("Postive for Covid19 ")
else:
    print("Good news, you have tested negative")

Postive for Covid19 
